<a href="https://colab.research.google.com/github/Kaguya2906/Chicago-City-Crime-Data-Retrieval-Platform/blob/main/http5008.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from elasticsearch import Elasticsearch
es = Elasticsearch(["http://localhost:9200"])

from flask import Flask, render_template, request

app = Flask("MyApp")

@app.route('/', methods=['GET', 'POST'])
def search():
    if request.method == 'POST':
        # Handle POST requests
        query = request.form.get('q')
        if not query:
            return render_template('search.html', results=None)
        else:
            body = {
                "query": {
                    "multi_match": {
                        "query": query,
                        "fields": ["Date", "Primary Type", "Location Description", "Block"]
                    }
                },
                "size": 100  # Set size to retrieve up to 10 hits
            }
            results = es.search(index='myindex', body=body)
            return render_template('results.html', query=query, results=results)
    else:
        # Handle GET requests
        return render_template('search.html', results=None)

import os

template_dir = os.path.join(os.getcwd(), 'templates')

if not os.path.exists(template_dir):
    os.makedirs(template_dir)

with open(os.path.join(template_dir, 'results.html'), 'w') as f:
    f.write('''
{% extends "base.html" %}

{% block content %}
  <h1>Search Results for "{{ query }}"</h1>
  {% if results.hits %}
    <table>
      <thead>
        <tr>
          <th>Date</th>
          <th>Primary Type</th>
          <th>Description</th>
          <th>Location Description</th>
          <th>Block</th>
          <th>Arrest</th>
        </tr>
      </thead>
      <tbody>
        {% for hit in results.hits.hits %}
          {% set source = hit._source %}
          <tr>
            <td>{{ source.Date }}</td>
            <td>{{ source['Primary Type'] }}</td>
            <td>{{ source['Description'] }}</td>
            <td>{{ source['Location Description'] }}</td>
            <td>{{ source['Block'] }}</td>
            <td>{{ source.Arrest }}</td>
          </tr>
        {% endfor %}
      </tbody>
    </table>
  {% else %}
    <p>No results found.</p>
  {% endif %}
{% endblock %}
''')

with open(os.path.join(template_dir, 'base.html'), 'w') as f:
    f.write('''
<!DOCTYPE html>
<html>
  <head>
    <title>{% block title %}{% endblock %}</title>
    <meta charset="UTF-8">
  </head>
  <body>
    {% block content %}{% endblock %}
  </body>
</html>
''')

if __name__ == '__main__':
    app.run(host='localhost', port=5008)

 * Serving Flask app "MyApp" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5008/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Apr/2023 21:56:42] "GET / HTTP/1.1" 200 -
/var/folders/fh/q6gx0j6d0d5f_5pw7h91mcxc0000gn/T/ipykernel_14801/3478150053.py:25: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  results = es.search(index='myindex', body=body)
/var/folders/fh/q6gx0j6d0d5f_5pw7h91mcxc0000gn/T/ipykernel_14801/3478150053.py:25: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  results = es.search(index='myindex', body=body)
127.0.0.1 - - [18/Apr/2023 21:56:46] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2023 21:56:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2023 21:56:55] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2023 21:57:01] "GE